This script will import and parse the entire SMILES database from the COD.

Those SMILES will then be compared to a testing set from OpenFF to define a test set
of CIF files (~100-300 crystals) for benchmarking.

The script will download matching cif files to the local machine.


In [2]:
# Import Block
import pandas as pd
import wget
import openff
import openbabel.pybel

In [3]:
# Import allcod.smi and the sage-train set

smiles = pd.read_csv('allcod.smi',names=['SMILES','COD ID'],sep='\t')
sage_train = pd.read_csv('sage-train-v1.csv')

In [6]:
# Make a list of SMILES from training set and prune duplicates
training_smiles = sage_train['Component 1'].drop_duplicates().to_frame()
training_smiles.rename(columns={'Component 1':'SMILES'},inplace=True)
combined_set = smiles.merge(training_smiles,how='left',on='SMILES',indicator=True)
matches = combined_set[combined_set['_merge'] == 'both']
print(matches['COD ID'])

215       1010439
359       1011023
360       1011024
374       1011063
3375      1504357
24122     2007371
34268     2100348
34269     2100349
34270     2100350
36279     2104943
38092     2200599
143548    4501702
143549    4501703
143550    4501704
207528    9009215
Name: COD ID, dtype: int64


In [5]:
qm_set = openff.evaluator.datasets.PhysicalPropertyDataSet.from_json('2-0-0-td-set-v1.json')
qm_set = qm_set['entries']['https://api.qcarchive.molssi.org:443/']
qm_smiles = []
for i in qm_set:
    foo = openbabel.pybel.readstring('can',i['cmiles'])
    qm_smiles.append(foo.write('smi').strip())
# Make a list of SMILES from qm set and prune duplicates
qm_training_smiles = pd.DataFrame(qm_smiles,columns=['SMILES'])
qm_combined_set = smiles.merge(qm_training_smiles,how='left',on='SMILES',indicator=True)
qm_matches = qm_combined_set[qm_combined_set['_merge'] == 'both']
print(qm_matches)
all_matches = matches.append(qm_matches)
print(all_matches['COD ID'])

AttributeError: module 'openff' has no attribute 'evaluator'